# ACS AWS Training: Day 5
## Nathan Miles
06/15/2020

<center><img src="./endgame.jpg" width="75%" height="75%"></center>

## Outline
- Docker 101
    - Containers
- Example
    - Create a lambda funciton that will,
        - run SExtractor on an image to identify all sources,
        - generate a catalog of photometry for all sources found in the image
    - Create a deployment package containing the Lambda function and its dependencies using `EC2`
        - `astropy`
        - `matplotlib`
        - `numpy`
        - `scipy`
        - `sep` (SExtractor in python)
    - Uploading the deployment package to S3
    - Building a Lambda function using deployment package
    - Testing the function in the console
    - Invoking the lambda function with `boto3`


## Docker
- Platform for containerizing applications
- Containers are standardized units of software
    - Packages up code and all its dependencies so it runs quickly and reliably in a variety computing environments
        - code, runtime, system tools, system libraries and settings
    - OS-level virtualization

<center><figure><img src="visual_representation_of_a_container.png" width="50%" height="50%" /><figcaption>Credit: <a href="https://www.docker.com/resources/what-container">What is a Container?</a></figcaption></figure></center>

## Example: Overview
- To circumvent any issues associated with installing/configuring Docker on your computer, we will build the deployment package on a small EC2 Instance.
- In the example we will,
    - build the deployment package on EC2,
    - upload it to S3,
    - create a Lambda function and specify the deployment package in S3 as the source code,
    - test our Lambda function in the AWS Lambda Console,
    - invoke our lambda function via `boto3`



## Example: Download the Software, Part 1
#### Place the files in the following directory: `~/acs_aws_day5_code`
- <a href="./day5_code/setup_ec2.sh" download>setup_ec2.sh</a>
    - Setup script from <a href="https://nmiles2718.github.io/presentations/acs_aws_day3/day3.html#/6">Day 3</a>
- <a href="./day5_code/build.sh" download>build.sh</a>
    - Build script used to create our deployment package
- <a href="./day5_code/requirements.txt" download>requirements.txt</a>
    - `pip` requirements file specifying the packages to install
- <a href="./day5_code/example_image_catalog.txt" download>example_image_catalog.txt</a>
    - Image catalog containing a list of images to be processed
- <a href="./day5_code/extract_photometry.py" download>extract_photometry.py</a>
    - Python class for performing aperture photometry using Python wrapped 
        - <a href="https://sep.readthedocs.io/en/v1.0.x/">sep.readthedocs.io</a>


## Example: Download the Software, Part 2
#### Place the files in the following directory: `~/acs_aws_day5_code`
- <a href="./day5_code/datavis.py" download>datavis.py</a>
    - Convenience class for handling all data visualization 
- <a href="./day5_code/aws_source_extraction.py" download>aws_source_extraction.py</a>
    - Python module for running source extraction on AWS
    - This is the file that contains the `handler` for the Lambda function
- <a href="./day5_code/run_photometry.py" download>run_photometry.py</a>
    - Python module for invoking the Lambda function via `boto3`
- <a href="./day5_code/launch_instance.py" download>launch_instance.py</a>
    - Launch an EC2 instance using a specified launch template and key pair

## Example: Building the deployment package, Part 1
- Change directory into `~/acs_aws_day5_code`
- Launch an EC2 template to use for creating the Docker container. 
    - For `TEMPATE_ID`, use `lt-0e86f881efd781a45`
    - For `KEYNAME` argument, exclude the `.pem` suffix 
    

``` console
python launch_instance.py -template_id <TEMPLATE_ID> -keyname <KEYNAME>
```

- Copy over the `acs_aws_day5_code` directory to the home directory (`~/`) of your EC2 instance

```console
scp -ri "path/to/key.pem" ~/acs_aws_day5_code ec2-user@public_dns:~
```

- Use `setup_ec2.sh` to upload your AWS access keys
    - For `KEYNAME`, include the `.pem` suffix
```console
bash setup_ec2.sh <EC2 Public DNS> <KEYNAME>
```


- Install and configure docker,
    - install: `sudo yum install docker`
    - start docker: `sudo service docker start`
    - set permissions: `sudo usermod -a -G docker ec2-user`


## Example: Building the deployment package, Part 2
- Log out of the EC2 instance
- Log back in and execute the command: `docker info`
    - If no output is shown, your docker installation failed
- Change directory to `~/acs_aws_day5_code`
- Execute the following commands to the build the `zip` file
    - `docker pull amazonlinux:2018.03`
        - Amazon Linux image that we use to build our deployment package in
    - `docker run -v $(pwd):/outputs -it amazonlinux:2018.03 /bin/bash /outputs/build.sh`
        - The deployment package is the `venv.zip` file
- While the docker command is running, create an S3 bucket
- Copy the deployment package from EC2 to S3
    - `aws s3 cp ./venv.zip s3://<YOUR BUCKET NAME>`
    - Replace `<YOUR BUCKET NAME>` with the name of the bucket you created in the previous step
- Navigate to your S3 bucket in the console and make sure your `zip` file was properly uploaded
    - Select the `venv.zip` file and click **"Copy path"**
        

## Example: Creating the Lambda function
- Navigate to the AWS Lambda Console
- Click **"Create Function"**
    - Function name: whatever you want
    - Runtime: `Python 3.6`
    - Choose or create an execution role:
        - Select **"Use an Existing Role"** --> `acs-aws-lambda`
- In the **Function code** section of the Lambda Console, select the **Actions** menu:
    - Choose **"Upload a file from Amazon S3"**
    - Paste the S3 path to your `venv.zip` file 
- Navigate to the **"Basic Settings"** block on the AWS Lambda console, click **"Edit"** and update the following,
    - Handler: `aws_source_extraction.handler`
    - Memory: 1600 MB
    - Timeout: 1 minute
    - Click **"Save"**

## Example: Configure a test event for the Lambda function
- Create a test event for your lambda function in the AWS Lambda console
- Copy and paste the test below making sure to update the "s3_output_bucket" to point to the bucket you created

```python
{
  "fits_s3_key": "testsdatangc104_crj.fits",
  "fits_s3_bucket": "test-data-day5",
  "radius": 3,
  "r_inner": 14,
  "r_outer": 16,
  "s3_output_bucket": <YOUR BUCKET NAME>
}
```

## Example: Invoke the lambda with `boto3`
- On your local machine, change directory to `~/acs_aws_day5_code`
- Execute the following,

```console
python run_photometry.py -catalog example_image_catalog.txt -lambda_name <YOUR LAMBDA NAME> -output_bucket <YOUR S3 BUCKET>
```